In [1]:
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')


### Normalize the effect values across all states and phases

In [2]:
def min_max_scale(df,data):
    data = df[df.columns.drop('state')]
    x = data.to_numpy()
    min_ = np.min(x)
    range_ = np.max(x) - np.min(x)
    normalized = x - min_ /range_
    return normalized

In [3]:
#convert p values to standard significance scales
def significance_values(list_):
    temp = []
    for element in list_:
        if element <= 0.001:
            temp.append("***")
        elif element > 0.001 and element <= 0.01:
            temp.append("**")
        elif element > 0.01 and element <= 0.05:
            temp.append("*")
        else:
            #incase the value is not significant
            temp.append("no")
    return temp



In [4]:
def map_plot(map_df, data_df, index, cmap, filename_,title,savedir='../../results/causal_impact/'):
    merged_df = pd.merge(left=map_df[['state', 'geometry']], right=data_df, how='left', on='state')
    merged_df.fillna(0,inplace=True)
    merged_df['center'] = merged_df['geometry'].apply(lambda x: x.representative_point().coords[:])
    merged_df['center'] = [coords[0] for coords in merged_df['center']]
    merged_df['p_center'] = [(item[0],item[1]-0.8) for item in merged_df['center']]
    fig, ax = plt.subplots(figsize=(12, 8))
    fig.patch.set_facecolor('white')

    merged_df.plot(column=index+' abs', cmap=cmap, linewidth=0.1, ax=ax, edgecolor='0.9', missing_kwds= dict(color = "lightgrey",))
    ax.axis('off')

    sm = plt.cm.ScalarMappable(cmap=cmap,norm=None)
    sm._A = []
    cbar = fig.colorbar(sm)
    
    # Annotate state names and significance values
    for idx, row in merged_df.iterrows():
        phase = index.split(" ")[1]
#         print(phase)
        pvalue_ = 'Phase '+str(phase)+' p'
        if row['Phase '+str(phase)+' p']!=0 and row['Phase '+str(phase)+' p']!="no":
            plt.annotate(s=row[pvalue_], xy=row['p_center'], horizontalalignment='center', size=6, alpha=1)
        plt.annotate(s=row['state'], xy=row['center'], horizontalalignment='center', size=6, alpha=1)
    if title=='Rt':
        plt.annotate("$R_t$ (Phase "+str(phase)+")",xy=(88,29), horizontalalignment='center', size=10, alpha=0.9)
    else:
        plt.annotate(str(title)+" (Phase "+str(phase)+")",xy=(88,29), horizontalalignment='center', size=10, alpha=0.9)
    fig.savefig(f'{savedir}/{index}_{filename_}.png', bbox_inches='tight', dpi=600)
    plt.close()


In [5]:
if __name__=='__main__':
    #read the shape file
    india_shp = 'maps/india/india.shp'
    india_df = geopandas.read_file(india_shp)   
    
    #get consistent state names:
    # Telangana?
    mapper = {
    'Andaman & Nicobar Island': 'Andaman and Nicobar Islands',
    'Arunanchal Pradesh': 'Arunachal Pradesh',
    'Dadara & Nagar Havelli': 'Dadra and Nagar Haveli',
    'Daman & Diu': 'Daman and Diu',
    'NCT of Delhi': 'Delhi',
    'Jammu & Kashmir':'Jammu & Kashmir and Ladakh'
    }
    india_df['state'] = india_df['ST_NM'].apply(lambda x: mapper[x] if x in mapper.keys() else x)
    
    # read the data about the absolute causal effects obtained through causal impact
    files_ = [
        'case_growth_effects_6_25.csv',
        'death_rate_effects_6_25.csv',
        'mobility_effects_6_25.csv',
        'rt_effects_6_25.csv'
    ]
    titles = ['Case Growth','Death Rate','Mobility','Rt']
    cmaps  = ['Reds','Oranges','Blues','Greens']
    directories = ['case_growth','death_rate','mobility','rt']
    for index_ in range(0,len(files_)):
        print(index_)
        data = pd.read_csv('../../results/causal_impact/'+str(files_[index_]),index_col=0)
        #separate lock and unlock phases
        unlock_data = data[['state','Phase 5 abs','Phase 6 abs']]
        lock_data = data[['state','Phase 1 abs','Phase 2 abs','Phase 3 abs','Phase 4 abs']]
        
        #obtain normalized data for lock period
        data_lock_norm = min_max_scale(lock_data,data)
        lock_data['Phase 1 abs'] = data_lock_norm[:,0]
        lock_data['Phase 2 abs'] = data_lock_norm[:,1]
        lock_data['Phase 3 abs'] = data_lock_norm[:,2]
        lock_data['Phase 4 abs'] = data_lock_norm[:,3]

        lock_data['Phase 1 p'] = data['Phase 1 p']
        lock_data['Phase 2 p'] = data['Phase 2 p']
        lock_data['Phase 3 p'] = data['Phase 3 p']
        lock_data['Phase 4 p'] = data['Phase 4 p']

        #obtain normalized data for unlock phase
        data_unlock_norm = min_max_scale(unlock_data,data)
        unlock_data['Phase 5 abs'] = data_unlock_norm[:,0]
        unlock_data['Phase 6 abs'] = data_unlock_norm[:,1]

        unlock_data['Phase 5 p'] = data['Phase 5 p']
        unlock_data['Phase 6 p'] = data['Phase 6 p']
        
        
        #convert p values to standard significant scale
        lock_data['Phase 1 p'] = significance_values(list(lock_data['Phase 1 p'].astype(float)))
        lock_data['Phase 2 p'] = significance_values(list(lock_data['Phase 2 p'].astype(float)))
        lock_data['Phase 3 p'] = significance_values(list(lock_data['Phase 3 p'].astype(float)))
        lock_data['Phase 4 p'] = significance_values(list(lock_data['Phase 4 p'].astype(float)))

        unlock_data['Phase 5 p'] = significance_values(list(unlock_data['Phase 5 p'].astype(float)))
        unlock_data['Phase 6 p'] = significance_values(list(unlock_data['Phase 6 p'].astype(float)))
    
        indices_lock = ['Phase 1','Phase 2','Phase 3','Phase 4']
        for i in indices_lock:
            map_plot(
                india_df,
                lock_data,
                i, 
                cmap=cmaps[index_],
                filename_ = titles[index_],
                title=titles[index_],
                savedir='../../results/causal_impact/'+str(directories[index_])
            )

        indices_unlock = ['Phase 5','Phase 6']
        for i in indices_unlock:
            map_plot(
                india_df,
                unlock_data,
                i, 
                cmap=cmaps[index_],
                filename_ = titles[index_],
                title=titles[index_],
                savedir='../../results/causal_impact/'+str(directories[index_])

            )




0
1
2
3
